The goal in this file is comparing the solution of regular Bounded-Makespan-Min-Cost with Fixed_x_Bounded_Makespan_Min_Cost and generate the figure automatically.

In [1]:
# importing packages
import numpy as np
import pandas as pd
from gurobipy import *


# from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
# from bokeh.plotting import figure, show
# from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS
# from bokeh.layouts import row, column, gridplot, widgetbox
# from bokeh.models.widgets import Panel, Tabs, Button

# from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import numpy as np
import pandas as pd

from gurobipy import *

# output_notebook()

In [2]:
ScnCount = [20, 50, 100, 200, 1000]
subScnCount = 20
maxScn = 1000

Number = range(13, 49)
SubNumber = [1]
directory2 = "./scenarioGame/10_scenarioGame_" 

In [3]:
# reading data
def readData(readNumber, readSubNumber):  
        jobs = 30
        testNumber = readNumber # 1 to 48
        subTestNumber = readSubNumber # 1 to 10
        fileName = 'J' + str(jobs) + str(testNumber) + '_' + str(subTestNumber) + '.RCP'
        file = '../dataRCPSP/A' + str(jobs) + '/j' + str(jobs) + 'rcp/' + fileName
        my_cols = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
        df = pd.read_table(file, sep='\t',delim_whitespace=True, engine='python', names=my_cols)
        duration = df.iloc[2:, 0].values
        resourcesUsage = df.iloc[2:, 1:5].values
        successorsCount = df.iloc[2:, 5].values
        successors = df.iloc[2:, 6:9].values
        resourcesCount = int(df.iloc[0, 1])
        jobsCount = int(df.iloc[0,0])
        availableResources = df.iloc[1,0:resourcesCount].values
        return duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources

In [4]:
# model sets and parameters for FCT model
def FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, scenarioCount, subScenarioCount):  
        activities = np.array(range(jobsCount))
        resources = np.array(range(resourcesCount))
        scenarios = np.array(range(scenarioCount))
        subScenarios = np.array(range(subScenarioCount))
        T = duration.sum()
        d = dict(zip(activities,resourcesUsage))
        M = 150
        N = availableResources.max()
        return activities, resources, d, T, M , N, scenarios, subScenarios

In [5]:
# scenario generating Uniform
def scenarioGen(scenarios, jobsCount, duration, activities):
    np.random.seed(0)
    durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    aMin = 1 - (12**0.5)/20 #10%
    aMax = 1 + (12**0.5)/20
    for s in scenarios:
        for a in activities:
            cr = np.random.rand()
            durationScenarios[a,s] = duration[a] * (aMin + cr * (aMax-aMin))
    p = dict(zip(activities,durationScenarios))
    return p



# # scenario generating Normal
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
#     for s in scenarios:
#         for a in activities:
#             if duration[a] != 0:
#                 cr = np.random.normal(duration[a], duration[a]*0.2)
#             else: cr = 0
#             if cr >= 0:
#                 durationScenarios[a,s] = cr
#             else: 
#                 durationScenarios[a,s] = duration[a]
#     p = dict(zip(activities,durationScenarios))
#     return p



# # scenario generating Left Skew 600
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
#     for a in activities[1:-1]:
#         rndsk = skewnorm.rvs(600, size=1000)
#         rndsk = rndsk * duration[a] *0.2 /rndsk.std()
#         rndsk = rndsk - rndsk.mean() + duration[a]
#         for s in scenarios:
#             durationScenarios[a,s] = rndsk[s]
    
#     p = dict(zip(activities,durationScenarios))
#     return p



# # scenario generating Left Skew 8
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
#     for a in activities[1:-1]:
#         rndsk = skewnorm.rvs(8, size=1000)
#         rndsk = rndsk * duration[a] *0.2 /rndsk.std()
#         rndsk = rndsk - rndsk.mean() + duration[a]
#         for s in scenarios:
#             durationScenarios[a,s] = rndsk[s]
    
#     p = dict(zip(activities,durationScenarios))
#     return p

In [6]:
# define variables of FCT model
def addFCTVars(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f, o, e

In [7]:
# define variables of FCT model
def addFCTVarsMM(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f

In [8]:
# define variables of FCT model
def addFCTVarsXX(activities, resources, T, availableResources):  
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return s, f, o, e

In [9]:
# constraint 2.5 of FCT model
def addFCTConst1(activities, successors, x):    
        FCT.addConstrs(
            (x[activityI, activityJ] 
             ==
             1
             for activityI in activities if activityI != activities[-1]
             for activityJ in activities[successors[activityI][successors[activityI] > 0].astype(np.int) - 1]), 
            name = "NetworkRelations")

In [10]:
# constraint 2.6a of FCT model
def addFCTConst2a(activities, x, s, p, M, scenarios):  
        FCT.addConstrs(
            (s[activityJ, scenario] -  s[activityI, scenario] 
             >=
             p[activityI][scenario] - M * (1 - x[activityI, activityJ])
             for scenario in scenarios
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [11]:
# constraint 2.6a of FCT model
def addFCTConst2aMM(activities, x, s, ppp, M):  
        FCT.addConstrs(
            (s[activityJ] -  s[activityI] 
             >=
             ppp[activityI] - M * (1 - x[activityI, activityJ])
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [12]:
# constraint 2.6b of FCT model
def addFCTConst2b(activities, s, o, scenarios):  
        FCT.addConstrs(
            (s[activity, scenario] 
             >=
             o[activity]
             for scenario in scenarios
             for activity in activities), 
            name = "NetworkStartTimeAndOrderRelations")

In [13]:
# constraint 2.7 of FCT model
def addFCTConst3(activities, x, f, resources, N):  
        FCT.addConstrs(
            (f[activityI, activityJ, resource] -  N * x[activityI, activityJ] 
             <=
             0
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]
             for resource in resources), 
            name = "NetworkFlowRelations")

In [14]:
# constraint 2.8 of FCT model
def addFCTConst4(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             d[activityI][resource]
             for activityI in activities if activityI != activities[0] 
             for resource in resources), 
            name = "OutgoingFlows")

In [15]:
# constraint 2.9 of FCT model
def addFCTConst5(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             d[activityJ][resource]
             for activityJ in activities if activityJ != activities[-1] 
             for resource in resources), 
            name = "IngoingFlows")

In [16]:
# constraint 2.10a of FCT model
def addFCTConst6(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activities[0], activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "FirstFlow")

In [17]:
# constraint 2.10b of FCT model
def addFCTConst7(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activityI, activities[-1], resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "LastFlow")

In [18]:
# constraint for bounding makespan
def addFCTConst9(activities, scenarios, scenarioCount, s, makespan):  
        FCT.addConstrs(
            (quicksum(s[activities[-1], scenario] for scenario in scenarios)
             <=
             makespan * scenarioCount
             for alaki in range(0,1)
             ), 
            name = "Makespandddd")

In [19]:
# Objective of FCT model -- min cost
def addFCTObj(activities, s, scenarios, scenarioCount, o): 
        obj = (quicksum(s[activity, scenario] - o[activity] 
                              for activity in activities
                              for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [20]:
# Objective of FCT model -- min makespan det
def addFCTObjMM(activities, s):
  
        obj = (s[activities[-1]])
        FCT.setObjective(obj, GRB.MINIMIZE)

In [21]:
# Objective of FCT model -- min expected makespan
def addFCTObjdW(activities, s, scenarios, scenarioCount):
  
        obj = (quicksum(s[activities[-1], scenario] for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [22]:
for i in Number:
    for j in SubNumber: 
        
        
        try:


            #MM##########################################################################################################
            ################################# MIN makespan for DETERMINISTIC
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN makespan for DETERMINISTIC    step=1  :   " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ############################################################################################################1

            makespanDet = 0
            makespanExp = 0

            duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources = readData(i, j)
            activities, resources, d, T, M , N, scenarios, subScenarios = \
                FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, maxScn, subScnCount)
            pAll = scenarioGen(scenarios, jobsCount, duration, activities)
            pMM = list(range(jobsCount))
            for act in range(jobsCount):
                pMM[act] = sum(pAll[act]) / maxScn
            FCT = Model('FCT: MIN makespan for DETERMINISTIC')
            FCT.setParam('TimeLimit', 60)
            x, s, f = addFCTVarsMM(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2aMM(activities, x, s, pMM, M)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjMM(activities, s)
            FCT.optimize()        

    #.............................................. Save MM-X

            directory = directory2 + str(i) + "_" + str(j) + "/"
            !mkdir $directory
            if FCT.Status == 2:

                # Open a file X
                xoName = directory+ "MM-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|MM|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()       


            #dW############################################################################################################
            ##################### MIN E(makespan) for SCENARIO base | XdW bede
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN E(makespan) for SCENARIO base | XdW bede   step=2  :  " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ##############################################################################################################2

            subP = dict(zip(activities,np.array(list(pAll.values()))[:,:subScnCount]))
            FCT = Model('FCT: MIN makespan for SCENARIO base')
            FCT.setParam('TimeLimit', 600)
            x, s, f, o, e = addFCTVars(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2a(activities, x, s, subP, M, subScenarios)
            addFCTConst2b(activities, s, o, subScenarios)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjdW(activities, s, subScenarios, subScnCount)
            FCT.optimize()        

    #.............................................. Save dW-X

            if FCT.Status == 2:
                # Open a file X
                xoName = directory + "dW-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            # Calculations
            if FCT.MIPGap <= 10000:
                aaa = 0 # Expected Makespan
                for vvv in range(subScnCount):
                    aaa += s[jobsCount - 1, vvv].X
            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|dW|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(aaa / subScnCount,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else: 
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()                 





            for scnCount in ScnCount:


                #SdW###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of dW shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  " + str(i) + "_" + str(j) + " || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################3

                fileName =  "dW-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)


                p = dict(zip(activities,np.array(list(pAll.values()))[:,:scnCount]))


                FCT = Model('for fixed X: MIN scenario used makespan | use x of dW')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName = "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SdW|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanExp = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



                #SMM###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of MM shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  " + str(i) + "_" + str(j) + "  || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################4

                fileName =   "MM-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)

                FCT = Model('for fixed X: MIN scenario used makespan | use x of MM')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SMM|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanDet = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



        except Exception as err:
            print("Erro {}".format(err))

iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_13_1
    MIN makespan for DETERMINISTIC    step=1  :   13_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 5109 rows, 5152 columns and 18495 nonzeros
Variable types: 4128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 1314 rows and 1322 columns
Presolve time: 0.09s
Presolved: 3795 rows, 3830 columns, 13993 nonzeros
Variable types: 3131 continuous, 699 integer (699 binary)

Root relaxation: objective 3.398569e+01, 2922 iterations, 0.37 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.98569    0   25          -   33.98569   

 12810  8834   34.23463   65   37  103.45879   34.23463  66.9%  60.5  290s
 13490  9240   34.23824   72   52  103.45879   34.23463  66.9%  64.4  295s
*13896  9064              97     101.8294635   34.23463  66.4%  66.1  296s
*13899  8754             100     101.8217184   34.23463  66.4%  66.1  296s
 14572  9077   34.23930   81   45  101.82172   34.23463  66.4%  68.5  300s
*14700  8804              89      99.4667669   34.23463  65.6%  69.4  300s
 15243  9059   34.30055   88   48   99.46677   34.23463  65.6%  71.4  305s
 16671  9833   39.22948  101   40   99.46677   34.23463  65.6%  76.2  311s
 17209 10114   34.35244  107   48   99.46677   34.23463  65.6%  78.4  315s
 17798 10435   34.35312  114   45   99.46677   34.23463  65.6%  80.4  320s
*18430 10427             179      99.0737041   34.23463  65.4%  82.5  322s
 18796 10623   34.38538  121   59   99.07370   34.23463  65.4%  83.6  327s
 19186 10859   34.38664  123   60   99.07370   34.23463  65.4%  83.9  330s
 21721 12068   34.41068  


Root relaxation: objective 4.364894e+01, 2472 iterations, 0.35 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   43.64894    0    7          -   43.64894      -     -    1s
     0     0   43.64894    0   25          -   43.64894      -     -    2s
     0     0   43.64894    0   22          -   43.64894      -     -    2s
     0     0   43.64894    0   11          -   43.64894      -     -    4s
     0     0   43.64894    0   18          -   43.64894      -     -    5s
     0     0   43.64894    0    9          -   43.64894      -     -    7s
     0     0   43.64894    0   10          -   43.64894      -     -    7s
     0     0   43.64894    0   15          -   43.64894      -     -    8s
     0     0   43.64894    0   16          -   43.64894      -     -    9s
     0     0   43.64894    0   10          -   43.64894      -     -    9s
     0     0   43.64894   

     0     0   45.91178    0   23          -   45.91178      -     -    1s
     0     0   45.91178    0   19          -   45.91178      -     -    1s
     0     0   45.91178    0   13          -   45.91178      -     -    1s
     0     2   45.91178    0   13          -   45.91178      -     -    2s
H 1414  1386                      86.8503451   45.91178  47.1%   116    3s
H 1510  1480                      77.7531162   45.91178  41.0%   114    3s
H 1544   312                      52.8725660   45.91178  13.2%   113    4s
  1791   386   45.91178   70   22   52.87257   45.91178  13.2%   120    5s
H 1800   387                      51.8793807   45.91178  11.5%   120    5s
H 1801   387                      51.8445052   45.91178  11.4%   120    5s
H 1832   386                      47.9228758   45.91178  4.20%   120    5s
* 2231   638              48      47.9160760   45.91178  4.18%   122    5s
H 2373   720                      47.8781843   45.91178  4.11%   120    5s
H 2690   847             

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [4e+01, 2e+02]
  RHS range        [8e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 19822 rows and 66530 columns
Presolve time: 0.09s
Presolved: 4138 rows, 2058 columns, 10494 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.222e+04
 Factor NZ  : 2.591e+05 (roughly 5 MBytes of memory)
 Factor Ops : 2.051e+07 (less than 1 second per iteration)
 Threads    : 22

Barrier performed 0 iterations in 0.13 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 701 iterations and 0.19 seconds
Optimal objective  4.739209862e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_15_1
  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  15_1 || 50
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeL

...

Showing barrier log only...

Presolve removed 138488 rows and 60722 columns
Presolve time: 0.53s
Presolved: 64212 rows, 13212 columns, 136616 nonzeros

Ordering time: 0.10s

Barrier statistics:
 AA' NZ     : 1.356e+06
 Factor NZ  : 7.145e+06 (roughly 90 MBytes of memory)
 Factor Ops : 9.917e+08 (less than 1 second per iteration)
 Threads    : 22

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.45482519e+02 -1.01708789e+04  4.60e+02 6.94e-18  1.61e+00     1s
   1   2.35290167e+02 -8.12669616e+03  1.44e+02 1.30e-02  3.67e-01     1s
   2   1.54022395e+02 -3.45231129e+03  2.38e+00 4.35e-15  4.41e-02     1s

Barrier performed 2 iterations in 1.13 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 5947 iterations and 1.36 seconds
Optimal objective  5.911379194e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_47_1
  for fixed X: MIN E(makespan) | use 

     0     0   63.03602    0   13  115.90517   63.03602  45.6%     -    0s
H    0     0                      73.0329272   63.03602  13.7%     -    0s
H    0     0                      71.9973424   63.03602  12.4%     -    0s
H    0     0                      63.0360191   63.03602  0.00%     -    0s
     0     0   63.03602    0   13   63.03602   63.03602  0.00%     -    0s

Cutting planes:
  Gomory: 4
  Implied bound: 7
  MIR: 12
  Flow cover: 14

Explored 1 nodes (3618 simplex iterations) in 0.41 seconds
Thread count was 24 (of 24 available processors)

Solution count 4: 63.036 71.9973 73.0329 115.905 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.303601910676e+01, best bound 6.303601910676e+01, gap 0.0000%
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
    MIN E(makespan) for SCENARIO base | XdW bede   step=2  :  48_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e

Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.36965356e+01  5.69465106e+01  1.67e+02 0.00e+00  2.20e-01     0s
   1   6.36965356e+01  5.62972847e+01  2.14e+01 4.86e-17  3.24e-02     0s
   2   6.36965356e+01  6.12486543e+01  2.04e+00 3.47e-17  4.39e-03     0s
   3   6.36965356e+01  6.31567246e+01  2.04e-01 2.78e-17  7.07e-04     0s
   4   6.36965356e+01  6.36855512e+01  2.17e-02 2.78e-17  1.63e-05     0s
   5   6.36965356e+01  6.36965155e+01  3.06e-05 1.39e-17  2.78e-08     0s
   6   6.36965356e+01  6.36965356e+01  3.27e-11 2.78e-17  2.86e-14     0s

Barrier solved model in 6 iterations and 0.13 seconds
Optimal objective 6.36965356e+01

Crossover log...

      28 DPushes remaining with DInf 0.0000000e+00                 0s
       0 DPushes remaining with DInf 0.0000000e+00                 0s

     212 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase comple

Optimal objective  6.438485138e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  48_1 || 200
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 202700 rows, 68128 columns and 396980 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 5e-03]
  Bounds range     [4e+01, 1e+02]
  RHS range        [8e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 202518 rows and 67691 columns
Presolve time: 0.35s
Presolved: 182 rows, 437 columns, 872 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.350e+02
 Factor NZ  : 2.241e+03
 Factor Ops : 3.169e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal  


Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.694e+04
 Factor NZ  : 1.366e+05 (roughly 23 MBytes of memory)
 Factor Ops : 1.565e+06 (less than 1 second per iteration)
 Threads    : 22

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.48696900e+04  7.86017952e+01  1.20e+00 7.29e+00  5.46e+00     3s
   1  -1.19365557e+04  1.40205105e+02  2.29e-01 9.36e+00  1.42e+00     3s
   2  -3.85707232e+03  1.40982989e+02  1.60e-16 9.07e-01  1.08e-01     3s
   3  -2.25088699e+02  1.40976085e+02  5.55e-17 7.23e-02  9.45e-03     4s
   4   3.17872661e+01  1.39521977e+02  5.55e-17 4.42e-03  2.76e-03     4s
   5   2.65213253e+01  1.22460564e+02  4.16e-17 1.09e-03  2.46e-03     4s
   6   3.58376017e+01  1.01523127e+02  2.08e-17 1.50e-04  1.68e-03     4s
   7   2.52528625e+01  8.74995779e+01  1.21e-17 8.10e-05  1.59e-03     4s
   8   5.66525220e+01  7.34357675e+01  7.59e-18 8.36e-06  4.30e-04     4s
   9   6.2

In [23]:
print("FINISHED")

FINISHED
